In [1]:
import json

def get_latest_game_run():
    import os
    from pathlib import Path
    
    # Get all directories in game_runs
    game_runs_dir = Path('../game_runs')
    run_dirs = [d for d in game_runs_dir.iterdir() if d.is_dir() and d.name.startswith('run_')]
    
    if not run_dirs:
        raise ValueError("No game runs found in game_runs directory")
    
    # Sort by directory name (which contains timestamp) in reverse order
    latest_run = sorted(run_dirs, reverse=True)[0]
    
    # Get the first game directory (assuming game_1)
    game_dir = next((d for d in latest_run.iterdir() if d.name.startswith('game_')), None)
    
    if not game_dir:
        raise ValueError(f"No game directory found in {latest_run}")
    
    return game_dir

# Usage example:
latest_game = get_latest_game_run()
llm_log_path = latest_game / 'root_logs' / 'llm_interactions.jsonl'

In [ ]:
def display_jsonl_fields(filepath):
    all_fields = set()
    
    with open(filepath, 'r') as f:
        for line in f:
            data = json.loads(line)
            # Add all top-level keys to the set
            all_fields.update(data.keys())
    
    print("\nFields found in JSONL entries:")
    print("-" * 40)
    for field in sorted(all_fields):
        print(f"- {field}")

# Usage
display_jsonl_fields(llm_log_path)

In [ ]:
def display_agent_api_keys(filepath):
    seen_combinations = set()
    with open(filepath, 'r') as f:
        for line in f:
            data = json.loads(line)
            combination = (data['agent_name'], data.get('api_key', 'No API key'))
            seen_combinations.add(combination)
    
    print("\nAgent and API key combinations:")
    print("-" * 40)
    for agent, api_key in sorted(seen_combinations):
        print(f"Agent: {agent}")
        print(f"API Key: {api_key}")
        print("-" * 40)

# Usage
display_agent_api_keys(llm_log_path)

In [ ]:
def display_last_conversation_by_agent(filepath, agent_name=None, api_key=None):
    # Dictionary to store the latest message for each agent+api_key combination
    latest_messages = {}
    
    with open(filepath, 'r') as f:
        for line in f:
            data = json.loads(line)
            current_agent = data['agent_name']
            current_api = data.get('api_key', 'No API key')
            
            # Skip if not matching the specified filters
            if (agent_name and current_agent != agent_name) or \
               (api_key and current_api != api_key):
                continue
                
            # Use tuple of agent and API key as dictionary key
            key = (current_agent, current_api)
            latest_messages[key] = data
    
    # Display the latest message for each combination
    for (agent, api), data in latest_messages.items():
        print(f"\n=== Latest Interaction for {agent} (API: {api}) ===")
        print(f"Timestamp: {data['timestamp']}\n")
        
        # Display messages
        for msg in data['messages']:
            role = msg['role'].upper()
            print(f"[{role}]:")
            print(f"{msg['content']}\n")
        
        # Display response
        print("[RESPONSE]:")
        print(f"{data['response']}\n")
        print("-" * 80)

# Usage examples:
# Display last conversation for all agents
display_last_conversation_by_agent(llm_log_path)